# MRC 데이터셋과 metrics에 익숙해지기

### Requirements



In [2]:
# !pip install datasets==1.4.1
# !pip install transformers==4.4.1

## 데이터셋 불러오기

In [1]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Dataset squad_kor_v1 downloaded and prepared to /opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7. Subsequent calls will reuse this data.


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

## 데이터셋 둘러보기

In [3]:
dataset["train"][0]

{'answers': {'answer_start': [54], 'text': ['교향곡']},
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'title': '파우스트_서곡'}

In [4]:
import random
pick = random.randint(0, len(dataset['train'])-1)
example = dataset['train'][pick]
example

{'answers': {'answer_start': [284], 'text': ['크리올']},
 'context': '링구아 프랑카(lingua franca)는 서로 다른 모어를 사용하는 화자들이 의사소통을 하기 위해 공통어(共通語)로 사용하는 제3의 언어(때로는 한 집단의 모어)를 말하며 국가나 단체에서 공식적으로 정한 언어를 뜻하는 공용어(公用語)와는 다른 개념이다. 정의를 통해 알 수 있듯, 링구아 프랑카는 특정 언어를 지칭하는 표현이 아니라, 언어 가교의 기능을 수행하는 언어들을 통칭하는 표현이다. 여기에서 의미가 파생되어 학술, 상업 등의 특정 분야에서 널리 사용되는 언어라는 뜻으로 사용되기도 한다. 피진이나 크리올은 링구아 프랑카와 유사한 개념이지만 정의상 완전히 일치하지는 않는다. (그러나 대다수가 언어 가교의 기능을 수행하기에 많은 부분 겹친다고 볼 수 있다.)',
 'id': '6498950-0-1',
 'question': '피진 처럼 링구아 프랑카와 유사한 개념으로는 무엇이 있습니까?',
 'title': '링구아_프랑카'}

`answer_start`는 context에서 answer가 시작하는 부분을 가리킨다.

In [5]:
context = example['context']
answer_text = example['answers']['text'][0]
answer_start = example['answers']['answer_start'][0]
answer_end = answer_start + len(answer_text)
print('Answer from context', context[answer_start:answer_end])

Answer from context 크리올


## 토크나이징 해보기

In [6]:
from transformers import AutoTokenizer

In [7]:
model_checkpoint = "bert-base-multilingual-cased"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

### cased vs. uncased

#### 1. lowercasing

**BERT uncased** : OpenGenus -> opengenus

**BERT cased** : OpenGenus

#### 2. accent striping

**BERT uncased** : OpènGènus -> opengenus

**BERT cased** : OpènGènus

#### 3. unicode normalization (NFD)

**BERT uncased** : 안녕 -> ㅇ, ㅏ, ㄴ, ㄴ, ㅕ, ㅇ

**BERT cased** : 안녕

In [9]:
example_text = dataset["train"][0]['question']
example_text

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'

토크나이징 결과를 살펴보기

In [10]:
tokenizer.tokenize(example_text)

['바',
 '##그',
 '##너',
 '##는',
 '괴',
 '##테',
 '##의',
 '파',
 '##우스',
 '##트를',
 '읽',
 '##고',
 '무',
 '##엇',
 '##을',
 '쓰',
 '##고',
 '##자',
 '했',
 '##는',
 '##가',
 '?']

토크나이징 & 인덱싱 & 어텐션 마스킹

In [11]:
tokenizer(example_text)

{'input_ids': [101, 9318, 78136, 70162, 11018, 8905, 119351, 10459, 9901, 89108, 101825, 9642, 11664, 9294, 119137, 10622, 9511, 11664, 13764, 9965, 11018, 11287, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer(example_text)['input_ids']

[101,
 9318,
 78136,
 70162,
 11018,
 8905,
 119351,
 10459,
 9901,
 89108,
 101825,
 9642,
 11664,
 9294,
 119137,
 10622,
 9511,
 11664,
 13764,
 9965,
 11018,
 11287,
 136,
 102]

역 토크나이징

In [13]:
' '.join(tokenizer.tokenize(example_text)).replace(' ##', '')

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가 ?'

In [14]:
tokenizer.decode(tokenizer(example_text)['input_ids'])

'[CLS] 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가? [SEP]'

## 전처리하기

In [15]:
max_seq_length = 384 # 질문과 컨텍스트, special token을 합한 문자열의 최대 길이
doc_stride = 128 # 컨텍스트가 너무 길어서 나눴을 때 오버랩되는 시퀀스 길이

샘플 전처리 해보기

In [16]:
examples = dataset['train'][:5]

In [17]:
tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # Truncate to max_length. This will only truncate the second sequence of a pair.
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True, # Whether or not to return overflowing token sequences.
        return_offsets_mapping=True,  # Whether or not to return (char_start, char_end) for each token.
        padding="max_length",
    )

In [18]:
tokenized_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [19]:
len(tokenized_examples['input_ids'])

10

In [20]:
tokenized_examples['input_ids'][0]

[101,
 9318,
 78136,
 70162,
 11018,
 8905,
 119351,
 10459,
 9901,
 89108,
 101825,
 9642,
 11664,
 9294,
 119137,
 10622,
 9511,
 11664,
 13764,
 9965,
 11018,
 11287,
 136,
 102,
 16221,
 10954,
 9318,
 78136,
 70162,
 11018,
 8905,
 119351,
 10459,
 9901,
 89108,
 15184,
 10622,
 62849,
 9642,
 11664,
 8924,
 8996,
 24974,
 10530,
 9246,
 32158,
 10739,
 8973,
 26737,
 35756,
 9448,
 36210,
 11261,
 9960,
 12424,
 90387,
 8907,
 79544,
 55670,
 10622,
 9511,
 26737,
 11018,
 9153,
 10622,
 8854,
 40410,
 119,
 9638,
 9485,
 12310,
 9318,
 78136,
 70162,
 11018,
 16347,
 27056,
 9387,
 9088,
 119267,
 11467,
 9407,
 16617,
 15891,
 54918,
 9056,
 100,
 9414,
 65649,
 10739,
 17342,
 9686,
 58931,
 11882,
 9489,
 89292,
 10530,
 8843,
 118813,
 51491,
 9272,
 97146,
 12605,
 26444,
 119394,
 100929,
 11513,
 9248,
 49742,
 9901,
 89108,
 15184,
 10459,
 9491,
 31720,
 10530,
 8896,
 105197,
 12490,
 11664,
 16139,
 119,
 19789,
 9901,
 46766,
 12424,
 9519,
 52015,
 77884,
 20308,
 1

In [21]:
tokenized_examples['offset_mapping']

[[(0, 0),
  (0, 1),
  (1, 2),
  (2, 3),
  (3, 4),
  (5, 6),
  (6, 7),
  (7, 8),
  (9, 10),
  (10, 12),
  (12, 14),
  (15, 16),
  (16, 17),
  (18, 19),
  (19, 20),
  (20, 21),
  (22, 23),
  (23, 24),
  (24, 25),
  (26, 27),
  (27, 28),
  (28, 29),
  (29, 30),
  (0, 0),
  (0, 4),
  (4, 5),
  (6, 7),
  (7, 8),
  (8, 9),
  (9, 10),
  (11, 12),
  (12, 13),
  (13, 14),
  (15, 16),
  (16, 18),
  (18, 19),
  (19, 20),
  (21, 23),
  (24, 25),
  (25, 26),
  (27, 28),
  (29, 30),
  (30, 31),
  (31, 32),
  (33, 34),
  (34, 35),
  (35, 36),
  (37, 38),
  (38, 39),
  (40, 42),
  (43, 44),
  (44, 45),
  (45, 46),
  (47, 48),
  (48, 49),
  (50, 53),
  (54, 55),
  (55, 56),
  (56, 57),
  (57, 58),
  (59, 60),
  (60, 61),
  (61, 62),
  (63, 64),
  (64, 65),
  (66, 67),
  (67, 69),
  (69, 70),
  (71, 72),
  (73, 74),
  (74, 75),
  (76, 77),
  (77, 78),
  (78, 79),
  (79, 80),
  (81, 85),
  (85, 87),
  (88, 89),
  (90, 91),
  (91, 92),
  (92, 94),
  (95, 96),
  (96, 97),
  (97, 98),
  (98, 100),
  (101, 1

In [22]:
tokenized_examples['overflow_to_sample_mapping']

[0, 0, 1, 1, 2, 2, 3, 3, 4, 4]

In [23]:
def prepare_train_features(examples):
    # 주어진 텍스트를 토크나이징 한다. 이 때 텍스트의 길이가 max_seq_length를 넘으면 stride만큼 슬라이딩하며 여러 개로 쪼갬.
    # 즉, 하나의 example에서 일부분이 겹치는 여러 sequence(feature)가 생길 수 있음.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # max_seq_length까지 truncate한다. pair의 두번째 파트(context)만 잘라냄.
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True, # 길이를 넘어가는 토큰들을 반환할 것인지
        return_offsets_mapping=True,  # 각 토큰에 대해 (char_start, char_end) 정보를 반환한 것인지
        padding="max_length",
    )
    
    # example 하나가 여러 sequence에 대응하는 경우를 위해 매핑이 필요함.
    overflow_to_sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # offset_mappings으로 토큰이 원본 context 내 몇번째 글자부터 몇번째 글자까지 해당하는지 알 수 있음.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # 정답지를 만들기 위한 리스트
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # 해당 example에 해당하는 sequence를 찾음.
        sequence_ids = tokenized_examples.sequence_ids(i)
        
        # sequence가 속하는 example을 찾는다.
        example_index = overflow_to_sample_mapping[i]
        answers = examples["answers"][example_index]
        
        # 텍스트에서 answer의 시작점, 끝점
        answer_start_offset = answers["answer_start"][0]
        answer_end_offset = answer_start_offset + len(answers["text"][0])

        # 텍스트에서 현재 span의 시작 토큰 인덱스
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        
        # 텍스트에서 현재 span 끝 토큰 인덱스
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # answer가 현재 span을 벗어났는지 체크
        if not (offsets[token_start_index][0] <= answer_start_offset and offsets[token_end_index][1] >= answer_end_offset):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # token_start_index와 token_end_index를 answer의 시작점과 끝점으로 옮김
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start_offset:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= answer_end_offset:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [27]:
sequences = prepare_train_features(dataset['train'][:5])
sequences[0], sequences[1]

(Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]))

## 문장이 나뉘는(길이 제한을 넘어서기 때문에) 예시 케이스

### 기존의 origin data를 보았을 때, 같은 context에 여러 개의 질문이 존재하는 것을 확인할 수 있다.

In [28]:
origin = dataset["train"][0]
origin['context'], origin['question']

('1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?')

In [29]:
origin = dataset["train"][1]
origin['context'], origin['question']

('1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?')

In [30]:
origin = dataset["train"][2]
origin['context'], origin['question']

('1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?')

### context 의 길이가 길어서, 1개의 질문이 2번 나오는 것을 알 수 있음.
- Data가 늘어난 것처럼 보인다. (전체 갯수는 증가하기 때문)
- Stride 만큼 second context 에서는 겹치는 부분이 생긴다.
- 남는 뒷부분에는 pad 를 부여한다.

In [31]:
exam = prepare_train_features(dataset['train'][:4])
tokenizer.decode(exam['input_ids'][0])

'[CLS] 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가? [SEP] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜 [SEP]'

In [32]:
tokenizer.decode(exam['input_ids'][1])

'[CLS] 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가? [SEP] 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [33]:
tokenizer.decode(exam['input_ids'][2])

'[CLS] 바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가? [SEP] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드 [SEP]'

In [51]:
tokenizer.decode(exam['input_ids'][3])

'[CLS] 바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가? [SEP] 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [28]:
tokenized_dataset = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)

# 평가 지표 계산하기

In [29]:
from datasets import load_metric

In [30]:
metric = load_metric('squad')

In [31]:
metric

Metric(name: "squad", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
   

In [32]:
example0 = dataset['train'][0]
example0

{'answers': {'answer_start': [54], 'text': ['교향곡']},
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'title': '파우스트_서곡'}

In [33]:
example1 = dataset['train'][1]
example1

{'answers': {'answer_start': [421], 'text': ['1악장']},
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'id': '6566495-0-1',
 'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?',
 'title': '파우스트_서곡'}

In [34]:
metric.compute(references=[{"id": example0["id"], "answers": example0["answers"]}], predictions=[{"id": example0["id"], "prediction_text": "교향곡"}])

{'exact_match': 100.0, 'f1': 100.0}

In [35]:
metric.compute(references=[{"id": example0["id"], "answers": example0["answers"]}], predictions=[{"id": example0["id"], "prediction_text": "교향곡 9번"}])

{'exact_match': 0.0, 'f1': 66.66666666666666}

In [36]:
metric.compute(references=[{"id": e["id"], "answers": e["answers"]} for e in [example0, example1]], predictions=[{"id": example0["id"], "prediction_text": "교향곡 9번"}, {"id": example1["id"], "prediction_text": "1악장"}])

{'exact_match': 50.0, 'f1': 83.33333333333333}